In [65]:
%load_ext autoreload
%autoreload 2

import numpy as np
import torch

from Generation.models import *
from transformers import (
    GPT2Config,
    GPT2Tokenizer
)
from transformers import AutoConfig

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [66]:
cache_dir='../HULK_new/Saved_models/'


#gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/Create_debate_DialoGPT-medium/'
gen_model_name_or_path='../HULK_new/Counterspeech/Saved_Models/Generator/Gab_DialoGPT-medium/'
#gen_model_name_or_path='microsoft/DialoGPT-small'

tokenizer = GPT2Tokenizer.from_pretrained(gen_model_name_or_path, cache_dir=cache_dir)
model = Model_Generation.from_pretrained(gen_model_name_or_path,cache_dir=cache_dir)
#model = model.to(device)
model = model.float()

gedi_model_name_or_path = '../HULK_new/Counterspeech/Saved_Models/Discriminator/Emotion_gedi_gpt2_fear/'
gedi_model = Model_Generation.from_pretrained(gedi_model_name_or_path,cache_dir=cache_dir)
#gedi_model.to(device)

KeyboardInterrupt: 

In [42]:
gedi_model_name_or_path = '../HULK_new/Counterspeech/Saved_Models/Discriminator/Emotion_gedi_gpt2_joy/'
gedi_model = Model_Generation.from_pretrained(gedi_model_name_or_path,cache_dir=cache_dir)

Loaded config and bias


In [63]:
params = {
    'sep_token':'<|endoftext|>',
    'max_generation_length': 60,
    'min_generation_length':20,
    'max_input_length':128,
    'unpertubed_count':0,
    'num_beams':1,
    'no_repeat_ngram_size':5,
    'repitition_penalty': 3.5,
    'control_type':'none',
    'k':100,
    'p':0.9,
    'filter_p':0.8,
    'target_p':0.8,
    'disc_weight':0.35,
    'class_bias':0,
    'sample':True,
    'temperature':1.0,
    'early_stopping':True,
    'model_path':'gpt2-medium',
    'dataset_hate':'CONAN',
    'task_name':[('Emotion','joy')],
    'coefficient':[4.5],
    'save_path': './../HULK_new/Counterspeech/Results/',
    'device': 'cuda',
    'batch_size':4,
    'cache_path':'./../HULK_new/Saved_models/',
    'generation_method':'huggingface',
}

In [64]:
prompt = "I hate all niggers <|endoftext|>"

start_len=0
text_ids = tokenizer.encode(prompt)
encoded_prompts=torch.LongTensor(text_ids).unsqueeze(0)
# multi_code = tokenizer.encode(secondary_code)
beam_outputs = model.generate(
            controller_alphas=None,
            controller_list=[gedi_model],
            control_type='gedi',
            positive_class='false',
            negative_class='true',
            tokenizer=tokenizer,
            class_bias=params['class_bias'],
            filter_p=params['filter_p'],
            target_p=params['target_p'],
            disc_weight=params['disc_weight'],
            unpertubed_count=params['unpertubed_count'],
            input_ids=encoded_prompts, 
            pad_token_id         = tokenizer.eos_token_id,
            max_length           = params['max_generation_length']+len(encoded_prompts[0]),
            min_length           = params['min_generation_length']+len(encoded_prompts[0]),
            top_k                = params["k"],
            top_p                = params["p"],
            repetition_penalty   = params["repitition_penalty"],
            temperature          = params["temperature"],
            num_beams            = params['num_beams'], 
            do_sample            = params['sample'],
            no_repeat_ngram_size = params['no_repeat_ngram_size'],  
            early_stopping       = params['early_stopping']
        )
reply = (tokenizer.decode(beam_outputs[0])).split(params['sep_token'])[1]
print("Prompt:",prompt)
print("Reply:",reply)

inside own function
GeDi estimates the probability that it sample is desired class is: 0.6015976667404175
Prompt: I hate all niggers <|endoftext|>
Reply: i hope you guys use the vote together and learn a lot about this new eugenic way of looking at history through using big labels will really do something for us all to look forward to seeing more positive changes


In [22]:
tokenizer.eos_token_id

50256

In [ ]:
encoded_prompts